# TF-IDF test

In [1]:
!nvidia-smi

Thu Sep 22 22:38:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   43C    P8    21W / 320W |   1070MiB / 10240MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pickle as pkl


import gensim.downloader as api
import numpy as np
import pandas as pd
from pyemd import emd
import spacy
# import wmd

from collections import Counter

from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.matutils import corpus2dense
# from sklearn.cluster import KMeans
# from spacy.language import Language
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.metrics import pairwise_distances
import cupy as cp
from cuml.metrics import pairwise_distances
from cuml.cluster import HDBSCAN, DBSCAN, KMeans

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_trf", disable=["tok2vec", "ner"])

In [4]:
!nvidia-smi

Thu Sep 22 22:39:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   44C    P8    18W / 320W |   3085MiB / 10240MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
df_ = pd.read_pickle('/home/miguel/Projects/tfm-nlp/data/interim/parsed_emails_chains_gt_1.pkl')
df = df_[df_['Chain_len'] == 16].dropna()

In [7]:
messages = df['Message'].dropna().str.lower().to_numpy()

In [8]:
def get_terms(string):
    # doc = nlp(string)
    terms = []
    for token in nlp(string):
        if not token.is_stop and not token.is_punct and not token.is_space\
        and not token.like_url and not token.like_email and not token.is_currency\
        and not token.like_num and token.pos_ != 'X' and not token.is_digit:
            terms.append(f'{token}_{token.pos_}_{token.lemma_}')
    return terms

In [9]:
%%time
terms = [get_terms(message) for message in messages]

CPU times: user 2.36 s, sys: 367 ms, total: 2.72 s
Wall time: 2.78 s


In [17]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Sep 22 23:58:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 56

In [ ]:
with open('/home/miguel/Projects/tfm-nlp/data/interim/tfidf/terms_lemma_gt_1.pkl', 'wb') as handle:
    pkl.dump(terms, handle, protocol=pkl.HIGHEST_PROTOCOL)

In [11]:
dictionary = Dictionary(terms)
corpus = [dictionary.doc2bow(term) for term in terms]
num_docs = dictionary.num_docs
num_terms = len(dictionary.keys())

In [12]:
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf_dense = corpus2dense(corpus_tfidf, num_terms, num_docs)
cupy_corpus_tfidf = cp.array(corpus_tfidf_dense.T)

In [13]:
dist_cosine = pairwise_distances(cupy_corpus_tfidf, metric='cosine')

In [14]:
len(cupy_corpus_tfidf)

112

In [15]:
dist_cosine.shape

(112, 112)

In [16]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Sep 22 22:40:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0

In [17]:
del dist_cosine

In [18]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Sep 22 22:29:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0

In [19]:
del cupy_corpus_tfidf

In [20]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Sep 22 22:36:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0